In [3]:
# импорт нужных библиотек
import os
from dotenv import load_dotenv
import requests as r
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn import objects as so

In [4]:
#API_URL='https://data-charts-api.hexlet.app'
#DATE_BEGIN='2023-03-01'
#DATE_END='2023-09-01'

In [5]:
load_dotenv()

DATE_BEGIN = os.getenv('DATE_BEGIN')
DATE_END = os.getenv('DATE_END')
API_URL = os.getenv('API_URL')

current_dir = os.getcwd() 
charts_folder = os.path.join(current_dir, 'charts')

if not os.path.exists(charts_folder):
    os.makedirs(charts_folder)

response_visits = r.get(f"{API_URL}/visits?begin={DATE_BEGIN}&end={DATE_END}")

json_data = response_visits.json()
df_visits = pd.DataFrame(json_data)

response_registrations = r.get(f"{API_URL}/registrations?begin={DATE_BEGIN}&end={DATE_END}")

json_data_registrations = response_registrations.json()
df_registrations = pd.DataFrame(json_data_registrations)
df_visits.describe()

MissingSchema: Invalid URL 'None/visits?begin=None&end=None': No scheme supplied. Perhaps you meant https://None/visits?begin=None&end=None?

In [ ]:
#для таблицы с посетителями
df_visits['datetime'] = pd.to_datetime(df_visits['datetime'])
#выбираем последнии посещения каждого пользователя
filtered_visits = df_visits.sort_values(by=['visit_id', 'datetime'], ascending=[True, False])
filtered_visits = filtered_visits.drop_duplicates(subset='visit_id', keep='first')
print(filtered_visits)

In [ ]:
#удаляем ботов
filtered_visits = filtered_visits.drop(filtered_visits[filtered_visits['platform'] == 'bot'].index)
print(filtered_visits)
#убираем время из даты
filtered_visits['date_group'] = pd.to_datetime(filtered_visits['datetime']).dt.date
filtered_visits['date_group'] = pd.to_datetime(filtered_visits['date_group'], format='%Y-%m-%d')
#удаляем ненужные столбцы
filtered_visits = filtered_visits.drop(columns=['datetime', 'user_agent'])

In [6]:
#для таблицы с регистрациями
#убираем время из даты
df_registrations['date_group'] = pd.to_datetime(df_registrations['datetime']).dt.date
df_registrations['date_group'] = pd.to_datetime(df_registrations['date_group'], format='%Y-%m-%d')
#удаляем ненужные столбцы
filtered_registrations = df_registrations.drop(columns=['datetime', 'email', 'registration_type'])

NameError: name 'df_registrations' is not defined

In [7]:
#по посещениям группируем по дате и платформе и считаем количество посетителей
visits_result = filtered_visits.groupby(['date_group', 'platform']).agg(visits = ('visit_id', 'count'))
visits_result = visits_result.reset_index()
print(visits_result)

NameError: name 'filtered_visits' is not defined

In [9]:
#по регистрациям группируем по дате и платформе и считаем количество регистраций
regs_result = filtered_registrations.groupby(['date_group', 'platform']).agg(registrations = ('user_id', 'count'))
regs_result = regs_result.reset_index()
print(regs_result)

NameError: name 'filtered_registrations' is not defined

In [10]:
#объединяем таблицы
res_conversion = visits_result.merge(regs_result, left_on=['date_group', 'platform'], right_on=['date_group', 'platform'])
#считаем конверсию
res_conversion['conversion'] = res_conversion['registrations'] / res_conversion['visits'] * 100
res_converstion = res_conversion.sort_values(by=['date_group'])
print(res_conversion)
#эксппортиуем результат в формате json
res_conversion.to_json('./conversion.json')

NameError: name 'visits_result' is not defined

In [11]:
#создаем новый датафрейм без расчета конверсии и данных по платформам
res_df = res_conversion.drop(columns = ['conversion'])
res_df = res_df.groupby(['date_group']).agg(visits=('visits', 'sum'), registrations=('registrations', 'sum'))
res_df = res_df.reset_index()
print(res_df)
#подгружаем данные о рекламе из csv файла
ads_df = pd.read_csv('./ads.csv')
#убираем время из даты
ads_df['date_group'] = pd.to_datetime(ads_df['date']).dt.date
ads_df['date_group'] = pd.to_datetime(ads_df['date_group'], format='%Y-%m-%d')
#удаляем ненужные столбцы
ads_df = ads_df.drop(columns=['date', 'utm_source', 'utm_medium'])

NameError: name 'res_conversion' is not defined

In [12]:
#объединяем таблицу посещений и регистраций с таблицой реклам
res_ads = res_df.merge(ads_df, left_on=['date_group'], right_on=['date_group'], how='left')
#проставляем 0 в стоимости рекламы в дни, когда ее не было
res_ads['cost'] =res_ads['cost'].fillna(0)
res_ads = res_ads.sort_values(by=['date_group'])
print(res_ads)
#экспортиуем результат в формате json
res_ads.to_json('./ads.json')

NameError: name 'res_df' is not defined

In [13]:
# сделаем датафрейм с указанием начала и конца рекламных кампаний
campaign_periods = ads_df.groupby(['utm_campaign']).agg(start = ('date_group', 'min'), end = ('date_group', 'max')).reset_index()
campaign_periods = campaign_periods[campaign_periods['start'] < DATE_END]
print(campaign_periods)

NameError: name 'ads_df' is not defined

In [14]:
ads_cost = res_ads.groupby(['utm_campaign']).agg(start = ('date_group', 'min'), end = ('date_group', 'max'), tot_cost = ('cost', 'sum'),
    tot_visits = ('visits', 'sum'), tot_regs = ('registrations', 'sum')).reset_index()
ads_cost['conv']= ads_cost['tot_regs'] / ads_cost['tot_visits'] * 100
print(ads_cost)

NameError: name 'res_ads' is not defined

In [15]:
#строим столбчатую диаграмму визитов с разбивкой по дате
plt.figure(num=None, layout='constrained',figsize=(25,10))
total_visits = sns.barplot(data=res_df, x='date_group', y='visits')
total_visits.bar_label(total_visits.containers[0], fontsize=10);
plt.title('Total Visits')
plt.xticks(rotation=45)
plt.show()
fig_total_visits = total_visits.get_figure()
fig_total_visits.savefig('./charts/visits_by_date.png')

NameError: name 'res_df' is not defined

<Figure size 2500x1000 with 0 Axes>

In [16]:
#строим столбчатую диаграмму визитов с разбивкой по дате и платформам
stacked_visits = res_conversion.drop(columns=['registrations', 'conversion'])
vis_by_platform = so.Plot(stacked_visits, x="date_group", y = 'visits', color="platform").add(so.Bar(), so.Stack()).layout(size=(15,5)).label(title='Visits by Platform (Stacked)')
vis_by_platform.save('./charts/visits_by_platform.png')

NameError: name 'res_conversion' is not defined

In [17]:
#строим столбчатую диаграмму регистраций с разбивкой по дате
plt.figure(num=None, layout='constrained',figsize=(25,10))
total_regs = sns.barplot(data=res_df, x='date_group', y='registrations')
total_regs.bar_label(total_regs.containers[0], fontsize=10);
plt.title('Total Registrations')
plt.xticks(rotation=45)
plt.show()
fig_total_regs = total_regs.get_figure()
fig_total_regs.savefig('./charts/registrations_by_date.png')

NameError: name 'res_df' is not defined

<Figure size 2500x1000 with 0 Axes>

In [18]:
#строим столбчатую диаграмму регистраций с разбивкой по дате и платформам
stacked_regs = res_conversion.drop(columns=['visits', 'conversion'])
regs_by_platform = so.Plot(stacked_regs, x="date_group", y = 'registrations', color="platform").add(so.Bar(), so.Stack()).layout(size=(15,5)).label(title='Registrations by Platform (Stacked)')
regs_by_platform.save('./charts/registrations_by_platform.png')

NameError: name 'res_conversion' is not defined

In [19]:
#подготовим данные для построения графика общей конверсии
overall_conversion = res_conversion.drop(columns=['platform', 'conversion'])
overall_conversion = overall_conversion.groupby('date_group').agg(visits = ('visits', 'sum'), registrations = ('registrations', 'sum'))
overall_conversion['conversion'] = overall_conversion['registrations'] / overall_conversion['visits'] * 100
overall_conversion = overall_conversion.drop(columns=['visits', 'registrations'])
#строим линейный график итоговой конверсии
plt.figure(figsize=(25,10))
plt.plot(overall_conversion.index, overall_conversion.values, marker='o', color='blue', label='Итоговая конверсия')
plt.title('Overall conversion')
plt.xlabel('Date')
plt.ylabel('Conversion(%)')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
for x,y in zip(overall_conversion.index, overall_conversion.values):
    plt.text(x, y, f'{round(y[0])}%', ha='left', va='bottom')
plt.tight_layout()
plt.savefig('./charts/overall_conversion.png')
plt.show()

NameError: name 'res_conversion' is not defined

In [20]:
#найдем все платформы
platforms = res_conversion['platform'].unique()

NameError: name 'res_conversion' is not defined

In [21]:
for p in platforms:
    #подготовка данных для построения графика
    platform_conversion = res_conversion[res_conversion['platform'] == p].drop(columns=['platform', 'visits', 'registrations'])
    platform_conversion = platform_conversion.groupby('date_group').sum()
    #строим линеный график конверсии по платформе
    plt.figure(figsize=(25,10))
    plt.plot(platform_conversion.index, platform_conversion.values, marker='o', color='green', label=p)
    plt.title(f'Conversion {p}')
    plt.xlabel('Date')
    plt.ylabel('Conversion(%)')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.legend()
    for x,y in zip(platform_conversion.index, platform_conversion.values):
        plt.text(x, y, f'{round(y[0])}%', ha='left', va='bottom')
    plt.tight_layout()
    plt.savefig(f'./charts/{p}_conversion.png')
    plt.show()

NameError: name 'platforms' is not defined

In [22]:
#подготовим данные для построения графика затрат на рекламу
ads_cost = res_ads.drop(columns=['visits', 'registrations', 'utm_campaign'])
ads_cost = ads_cost[ads_cost['cost'] > 0]
ads_cost = ads_cost.groupby('date_group').sum()
#строим линейный график затрат на рекламу
plt.figure(figsize=(25,10))
plt.plot(ads_cost.index, ads_cost.values, marker='o', color='blue')
plt.title('Aggregated Ad Campaign Costs (by day)')
plt.xlabel('Date')
plt.ylabel('Cost(RUB)')
plt.xticks(rotation=45)
plt.grid(True)
for x,y in zip(ads_cost.index, ads_cost.values):
    plt.text(x, y, f'{round(y[0])}RUB', ha='left', va='bottom')
plt.tight_layout()
plt.savefig('./charts/ads_cost.png')
plt.show()

NameError: name 'res_ads' is not defined

In [23]:
#построим график посещений в дни рекламных кампаний
plt.figure(figsize=(25,10))
campaign_colors = dict(zip(campaign_periods['utm_campaign'].unique(), plt.cm.tab10.colors[:len(campaign_periods['utm_campaign'].unique())]))
plt.subplot(2, 1, 1)
plt.plot(res_ads['date_group'], res_ads['visits'], marker='o', color='black', label='Visits')
plt.axhline(y=np.nanmean(res_ads.visits), linestyle='dashed', color='green', label='Average Number of Visits')
for _,row in campaign_periods.iterrows():
    plt.axvspan(row['start'], row['end'], label=row['utm_campaign'], color=campaign_colors[row['utm_campaign']], alpha=0.5)
plt.title('Visits during marketing active dates')
plt.xlabel('Date')
plt.ylabel('Unique Visits')
plt.xticks(rotation=45)
plt.grid(True)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())
plt.savefig('./charts/ads_visits.png')
plt.show()

NameError: name 'campaign_periods' is not defined

<Figure size 2500x1000 with 0 Axes>

In [24]:
#построим график регистраций в дни рекламных кампаний
plt.figure(figsize=(25,10))
campaign_colors = dict(zip(campaign_periods['utm_campaign'].unique(), plt.cm.tab10.colors[:len(campaign_periods['utm_campaign'].unique())]))
plt.subplot(2, 1, 1)
plt.plot(res_ads['date_group'], res_ads['registrations'], marker='o', color='green', label='Registrations')
plt.axhline(y=np.nanmean(res_ads.registrations), linestyle='dashed', color='olive', label='Average Number of Registrations')
for _,row in campaign_periods.iterrows():
    plt.axvspan(row['start'], row['end'], label=row['utm_campaign'], color=campaign_colors[row['utm_campaign']], alpha=0.5)
plt.title('Registrations during marketing active dates')
plt.xlabel('Date')
plt.ylabel('Registrations')
plt.xticks(rotation=45)
plt.grid(True)
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())
plt.savefig('./charts/ads_registrations.png')
plt.show()

NameError: name 'campaign_periods' is not defined

<Figure size 2500x1000 with 0 Axes>